In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import sympy as sp
import numpy as np
from scipy.constants import mu_0, epsilon_0
from sympy.physics.units import *

In [3]:
sp.init_printing()

In [4]:
# geometric variables
# independent variables
#coil outter diameter [cm], coil inner diameter [cm], trace width, core width, pcb thickness, 
# airgap length initial [cm], airgap final initial [cm],  core thickness, copper thickness, coil pitch (spacing between traces)
d_out, d_in, w, g, d, l_g_0, l_g_1, t_c, t_b, p = sp.symbols("d_out, d_in, w, g, d, l_g_0, l_g_1, t_c, t_b, p")
# initial design constraints (geometric) distances are all in millimeters
d_out = 1.0     * cm
d_in  = 0.04514 * cm # ceil(0.0254) + 0.01524
w     = 0.01524 * cm
p     = 0.01524 * cm
d     = 0.16    * cm #oshpark board thickness
l_g_0 = 0.3     * cm # unpressed switch
l_g_1 = (0.1 + 0.00254) * cm # pressed switch, with oshpark soldermask thickness, and protective sheet
g     = 1.1     * cm
t_c   = 0.035   * cm # web searches
t_b   = 0.00356 * cm
# dependent variables
#mpl, A_c, mtl, A_t, n = sp.symbols("mpl, A_c, mtl, A_t, n")
# mean path length [cm]
mpl_1 = 2*((1/2)*(g + d_out)+ (1/3)*(t_c + t_b + l_g_1))
mpl_0 = 2*((1/2)*(g + d_out)+ (1/3)*(t_c + t_b + l_g_0))
# mean turn length [mm]
mtl = (np.pi/2)*(d_out + d_in)
# core effective area [cm**2]
A_c = (np.pi/4)*(g**2)
# trace area [mm**2]
n = sp.floor(((d_out - 1/2*w) - (d_in + 1/2*w))/(2*(w + p)))
A_t = mtl*w*n

# electrical variables
# independent variables
# material permitivity and permiability
eps, mu = sp.symbols("eps, mu")
eps   = 4.6 * (pico*farads) / mm  #oshpark
mu    = 1500  #* (newton / ampere**2) #silicon steel laminate sheet
# dependent variables
#C_p, R_t, L, F = sp.symbols("C_p, R_t, L, F")
# paracitic capacitance [pF], assuming a plane on opposite side of pcb
C_p = 0.0085*(eps)*(A_t*100*mm**2 / cm**2)/(d*10*mm/cm)
# fringing factor
Fr_0 = (1 + (l_g_0/sp.sqrt(A_c))*sp.ln(2*g/l_g_0))
Fr_1 = (1 + (l_g_1/sp.sqrt(A_c))*sp.ln(2*g/l_g_1))
# trace resistance [uOhms/mM]
R = (mtl*10*mm/cm)*n*(10**(-6)) * (micro * ohms)/mm
# self inductance with fringing effect [H]
L_0 = Fr_0*(0.4*np.pi*(n**2)*A_c*(10**(-9)))/(l_g_0 + (mpl_0/mu)) * H / cm
L_1 = Fr_1*(0.4*np.pi*(n**2)*A_c*(10**(-9)))/(l_g_1 + (mpl_1/mu)) * H / cm

# constraining equations
# independent variables [A], [V]
i_dc, i_ac, v_dc, v_ac = sp.symbols("i_dc, i_ac, v_dc, v_ac")
# initial design constraints (electric)
# dependent variables
#B_ac, B_dc, B_pk, I_pk, V_min = sp.symbols("B_ac, B_dc, B_pk, I_pk, V_min")
# ac, dc, and peak B-fields, [Tesla]
#B_ac = (0.4*pi*n*(i_ac/2)*(10**(-4))) / (l_g + (mpl/mu))
#B_dc = (0.4*pi*n*i_dc*(10**(-4))) / (l_g + (mpl/mu))
#B_pk = B_dc + (B_ac/2)
#i_pk = i_dc + (i_ac/2)

# this main issue with design is the correct choice of geometry, current, and frequency to ensure a large enough detectable
# change in inductance, having an airgap that varies.


## in ---- R ---- L ---- out
##  |                     |
##  | ---- C_p -----------|


In [5]:
print(L_0, L_1)
print(R)
print(C_p)
print(n)

1.43741567552195e-6*henry 3.41677831647223e-6*henry
2.46255310947963e-10*ohm
9.17122498180724e-13*farad
15
